 #  The Battle of Neighborhoods Capstone - San Francisco

* Install/import the necessary python libraries for clustering, map plotting and data cleaninh

In [105]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
# !conda install -c conda-forge folium=0.5.0
import folium #map plotting
#importing matplotlib and necessary modules
import matplotlib.cm as cm
import matplotlib.colors as colors

* Get initial look at zip code data for San Francisco

In [137]:
response = requests.get("http://www.healthysf.org/bdi/outcomes/zipmap.htm")
soup = BeautifulSoup(response.text, "lxml")
table = soup.find_all("table")
df = pd.read_html(str(table))
df = pd.DataFrame(df[4])
df.columns = df.iloc[0]
# Sort neighborhoods by population
df.sort_values(by=['Population (Census 2000)'], inplace=True)
# print(df)
# We see that Chinatown is the smallest neighborhood and Inner Mission/Bernal Heights  is the largest neighborhood
df = df.iloc[1:-1, :-1]
df_sf = df
df_sf.head()

,Zip Code,Neighborhood
3,94107,Potrero Hill
17,94127,St. Francis Wood/Miraloma/West Portal
15,94123,Marina
2,94103,South of Market
19,94132,Lake Merced


In [138]:
!pip install uszipcode
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in df_sf.iterrows():
    zipcode = search.by_zipcode(row["Zip Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

df_sf["Latitude"] = latitude
df_sf["Longitude"] = longitude

# Drop any null values
df_sf.dropna(inplace=True)
print(df_sf.isnull())

0   Zip Code  Neighborhood  Latitude  Longitude
3      False         False     False      False
17     False         False     False      False
15     False         False     False      False
2      False         False     False      False
19     False         False     False      False
20     False         False     False      False
18     False         False     False      False
1      False         False     False      False
8      False         False     False      False
9      False         False     False      False
16     False         False     False      False
11     False         False     False      False
12     False         False     False      False
21     False         False     False      False
13     False         False     False      False
10     False         False     False      False
14     False         False     False      False
5      False         False     False      False
7      False         False     False      False
6      False         False     False    

* An initial glance at San Francisco

In [139]:
latitude = 37.7792808
longitude = -122.4192363
sf_map = folium.Map(location = [latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_sf['Latitude'], df_sf['Longitude'], df_sf['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(sf_map)  
    
sf_map

### Clustering Neighborhoods in SF

* Get FourSquare API credentials

In [140]:
CLIENT_ID = '1ETHRMMH2RJA3VQRGIXOEMF5TRYVL55AI1I432GTOAWWXGKP' 
CLIENT_SECRET = 'IYX1A0RDTDJV2RNAUYED2G4MCOWEIHLYGYS1RRBM3BPEUGNT' 
VERSION = '20180604'
LIMIT = 50

In [141]:
# Abstract code for grabbing venues for the different neighborhoods in SF using the API

In [142]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # GET request for venues
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [155]:
# Comparing the smallest neighborhoods with other possibly similar neighborhoods and the largest neighborhood
df_ch = df_sf[df_sf['Neighborhood'].str.contains('Chinatown') | df_sf['Neighborhood'].str.contains('North Beach/Chinatown') | 
              df_sf['Neighborhood'].str.contains('Western Addition/Japantown') | df_sf['Neighborhood'].str.contains('Heights') 
              | df_sf['Neighborhood'].str.contains('Hill') | df_sf['Neighborhood'].str.contains('Market')]

sf_venues = getNearbyVenues(names = df_ch['Neighborhood'],
                                   latitudes = df_sf['Latitude'],
                                   longitudes = df_sf['Longitude'])
                                  
sf_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Potrero Hill,37.77,-122.39,SPARK Social SF,37.770762,-122.391689,Street Food Gathering
1,Potrero Hill,37.77,-122.39,Stagecoach Greens,37.770867,-122.390261,Mini Golf
2,Potrero Hill,37.77,-122.39,Spro Coffeelab,37.770835,-122.391400,Coffee Shop
3,Potrero Hill,37.77,-122.39,SFFSoccer Mission Bay Field,37.770886,-122.392197,Soccer Field
4,Potrero Hill,37.77,-122.39,Señor Sisig,37.770809,-122.391437,Food Truck


In [156]:
sf_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Hayes Valley/Tenderloin/North of Market,50,50,50,50,50,50
Inner Mission/Bernal Heights,50,50,50,50,50,50
North Beach/Chinatown,50,50,50,50,50,50
Parkside/Forest Hill,50,50,50,50,50,50
Polk/Russian Hill (Nob Hill),16,16,16,16,16,16
Potrero Hill,50,50,50,50,50,50
South of Market,4,4,4,4,4,4
Western Addition/Japantown,17,17,17,17,17,17


In [157]:
# use one hot encoding for categorical variables (Venue Category)
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
sf_onehot['Neighborhood'] = sf_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

sf_onehot.head()

,Neighborhood,Alternative Healer,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,...,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Tiki Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Potrero Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Potrero Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Potrero Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Potrero Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Potrero Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped.head()

,Neighborhood,Alternative Healer,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,...,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Tiki Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Hayes Valley/Tenderloin/North of Market,0.00,0.0200,0.02,0.00,0.02,0.00,0.04,0.0,0.04,...,0.00,0.02,0.0000,0.04,0.00,0.00,0.06,0.02,0.00,0.0000
1,Inner Mission/Bernal Heights,0.00,0.0000,0.00,0.02,0.00,0.00,0.04,0.0,0.00,...,0.00,0.00,0.0200,0.04,0.02,0.04,0.02,0.04,0.02,0.0000
2,North Beach/Chinatown,0.02,0.0200,0.00,0.00,0.02,0.02,0.00,0.0,0.00,...,0.02,0.00,0.0400,0.00,0.00,0.00,0.00,0.06,0.00,0.0200
3,Parkside/Forest Hill,0.02,0.0200,0.00,0.00,0.02,0.02,0.00,0.0,0.00,...,0.02,0.00,0.0400,0.00,0.00,0.00,0.00,0.06,0.00,0.0200
4,Polk/Russian Hill (Nob Hill),0.00,0.0625,0.00,0.00,0.00,0.00,0.00,0.0,0.00,...,0.00,0.00,0.0625,0.00,0.00,0.00,0.00,0.00,0.00,0.0625


In [167]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [168]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        # append 'st', 'nd', 'rd' to the top 3 venues
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hayes Valley/Tenderloin/North of Market,Coffee Shop,Vietnamese Restaurant,Sandwich Place,Restaurant,Café,Bar,Pizza Place,Bakery,Food Truck,Theater
1,Inner Mission/Bernal Heights,Performing Arts Venue,French Restaurant,Indian Restaurant,Sushi Restaurant,Cocktail Bar,Bakery,Concert Hall,Wine Bar,Theater,Vegetarian / Vegan Restaurant
2,North Beach/Chinatown,French Restaurant,Wine Bar,Gym / Fitness Center,Italian Restaurant,Salad Place,Thai Restaurant,Deli / Bodega,Japanese Restaurant,Electronics Store,Hotpot Restaurant
3,Parkside/Forest Hill,French Restaurant,Wine Bar,Gym / Fitness Center,Italian Restaurant,Salad Place,Thai Restaurant,Deli / Bodega,Japanese Restaurant,Electronics Store,Hotpot Restaurant
4,Polk/Russian Hill (Nob Hill),Park,Library,Playground,Coffee Shop,Dog Run,Café,Food,Mexican Restaurant,Burger Joint,Bookstore


In [169]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3
sf_grouped_clustering = sf_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(sf_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 2, 0], dtype=int32)

In [170]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sf_merged = df_sf
sf_merged = sf_merged.merge(neighborhoods_venues_sorted, on = 'Neighborhood')

sf_merged.head()

,Zip Code,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94107,Potrero Hill,37.77,-122.39,1,Food Truck,Coffee Shop,Pharmacy,Pizza Place,Bank,Café,Street Food Gathering,Yoga Studio,Mini Golf,Playground
1,94103,South of Market,37.78,-122.41,2,Park,Bus Line,Scenic Lookout,Fountain,Yoga Studio,Food,Cupcake Shop,Dance Studio,Deli / Bodega,Design Studio
2,94133,North Beach/Chinatown,37.80,-122.44,1,French Restaurant,Wine Bar,Gym / Fitness Center,Italian Restaurant,Salad Place,Thai Restaurant,Deli / Bodega,Japanese Restaurant,Electronics Store,Hotpot Restaurant
3,94102,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,1,Coffee Shop,Vietnamese Restaurant,Sandwich Place,Restaurant,Café,Bar,Pizza Place,Bakery,Food Truck,Theater
4,94115,Western Addition/Japantown,37.79,-122.44,0,Pizza Place,Park,Gym,Café,Sandwich Place,Snack Place,Performing Arts Venue,Coffee Shop,Cocktail Bar,Mexican Restaurant


In [163]:

# create map
latitude = 37.7792808
longitude = -122.4192363
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

In [171]:
# Exploring the clusters formed to verify our initial assumption

In [164]:
sf_merged.loc[sf_merged['Cluster Labels'] == 1, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Potrero Hill,Food Truck,Coffee Shop,Pharmacy,Pizza Place,Bank,Café,Street Food Gathering,Yoga Studio,Mini Golf,Playground
2,North Beach/Chinatown,French Restaurant,Wine Bar,Gym / Fitness Center,Italian Restaurant,Salad Place,Thai Restaurant,Deli / Bodega,Japanese Restaurant,Electronics Store,Hotpot Restaurant
3,Hayes Valley/Tenderloin/North of Market,Coffee Shop,Vietnamese Restaurant,Sandwich Place,Restaurant,Café,Bar,Pizza Place,Bakery,Food Truck,Theater
5,Parkside/Forest Hill,French Restaurant,Wine Bar,Gym / Fitness Center,Italian Restaurant,Salad Place,Thai Restaurant,Deli / Bodega,Japanese Restaurant,Electronics Store,Hotpot Restaurant
6,Polk/Russian Hill (Nob Hill),Park,Library,Playground,Coffee Shop,Dog Run,Café,Food,Mexican Restaurant,Burger Joint,Bookstore
7,Inner Mission/Bernal Heights,Performing Arts Venue,French Restaurant,Indian Restaurant,Sushi Restaurant,Cocktail Bar,Bakery,Concert Hall,Wine Bar,Theater,Vegetarian / Vegan Restaurant


In [165]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,South of Market,Park,Bus Line,Scenic Lookout,Fountain,Yoga Studio,Food,Cupcake Shop,Dance Studio,Deli / Bodega,Design Studio


In [172]:
# We ran our KMeans algorithm with k=3 for the number of clusters but we find that our 3rd cluster is empty


In [173]:
sf_merged.loc[sf_merged['Cluster Labels'] == 3, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [174]:
# Since San Francisco only has 21 Neighborhoods and we further limited the number of neighborhoods in our analysis, 
#it is understandable that our 3rd cluster would be reasonably empty